In [1]:
from chatterer import BaseMessage, Chatterer, HumanMessage
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# from utils import ainvoke_graph, astream_graph

llm = Chatterer.from_provider(
    "openai:gpt-5-nano",
    option={
        "kwargs": {
            "reasoning_effort": "minimal",
        },
    },
)


async with MultiServerMCPClient({
    "test": {
        "url": "http://localhost:8000/mcp",
        "transport": "streamable_http",
    }
}).session("test") as session:
    tools = await load_mcp_tools(session)
    for tool in tools:
        print(f"- [Tool] name: {tool.name}, description: {tool.description}, args: {tool.args}")
    agent = create_agent(llm.client, tools)
    result = await agent.ainvoke({
        "messages": [
            HumanMessage(content="현재 서울 날씨가 어떤가?"),
        ]
    })
    for message_no, message in enumerate(result["messages"]):
        message: BaseMessage
        print(f"----------\nMsg [{message_no + 1}]\n{message.model_dump_json(indent=4)}")
        


- [Tool] name: add, description: Adds two integer numbers together., args: {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}
- [Tool] name: get_weather, description: Get the weather of the city., args: {'city': {'type': 'string'}}
- [Tool] name: search_products, description: Search the product catalog., args: {'query': {'type': 'string'}, 'max_results': {'default': 10, 'type': 'integer'}, 'sort_by': {'default': 'relevance', 'type': 'string'}, 'category': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None}}
----------
Msg [1]
{
    "content": "현재 서울 날씨가 어떤가?",
    "additional_kwargs": {},
    "response_metadata": {},
    "type": "human",
    "name": null,
    "id": "d356aae2-960f-4451-8eeb-e9d4023ca9ba"
}
----------
Msg [2]
{
    "content": "",
    "additional_kwargs": {
        "refusal": null
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 24,
            "prompt_tokens": 210,
            "total_tokens": 234,
            "c